In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import datasets, transforms
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.utils.data as data
import os
import pandas as pd
#from PIL import Image


In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)
batchNum = 64

cuda:0


# dataloader

In [1]:
TEST_DATA_PATH = "./testdata"

In [2]:
TRAIN_DATA_PATH = "./traindata"

In [5]:
# data transform, you can add different transform methods 
img_size = 224
transform_bright = [transforms.ColorJitter(brightness=0.1, contrast=0.2, saturation=0, hue=0),
                    transforms.RandomRotation(50, expand=True)]
transform_set = [ 
             
             transforms.RandomPerspective(distortion_scale=0.3, p=0.5),
             transforms.RandomHorizontalFlip(p=0.3),
             transforms.RandomHorizontalFlip(p=0.3)]

train_transform = transforms.Compose([transforms.RandomChoice(transform_bright),
                                      transforms.RandomOrder(transform_set),
                                      transforms.Resize(img_size),
                                      transforms.ToTensor()])

dataset = torchvision.datasets.ImageFolder(TRAIN_DATA_PATH,
                                            transform=train_transform
                                            )
TOTAL_SIZE = len(dataset)



# spilt your data into train and val
ratio = 0.9
train_len = round(TOTAL_SIZE * ratio)
valid_len = round(TOTAL_SIZE * (1-ratio))

train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_len, valid_len])
# train_dataset = train_dataset.to(device)
# val_dataset = val_dataset.to(device)
#print(len(train_dataset))
#print(len(val_dataset))
# your setting
train_dataset = dataset

train_data_loader = data.DataLoader(train_dataset, batch_size=batchNum, shuffle=True, num_workers=2)

val_data_loader = data.DataLoader(val_dataset, batch_size=batchNum, shuffle=False, num_workers=2)
print(dataset)
print(dataset.class_to_idx)
pallet = [5,2,4,0,1,3]

Dataset ImageFolder
    Number of datapoints: 521
    Root location: ./traindata
    StandardTransform
Transform: Compose(
               RandomChoice(
               ColorJitter(brightness=[0.9, 1.1], contrast=[0.8, 1.2], saturation=None, hue=None)
               RandomRotation(degrees=[-50.0, 50.0], interpolation=nearest, expand=True, fill=0)
           )
               RandomOrder(
               RandomPerspective(p=0.5)
               RandomHorizontalFlip(p=0.3)
               RandomHorizontalFlip(p=0.3)
           )
               Resize(size=224, interpolation=bilinear)
               ToTensor()
           )
{'airplane': 0, 'bird': 1, 'car': 2, 'cat': 3, 'dog': 4, 'horse': 5}


In [6]:
# I have written the function for you this time, but it's strongly recommended that you 
# understand how to do training and validation


def train(model, data_loader, optimizer, epoch, loss, verbose=True):
    model.train()
    loss_avg = 0.0
    for batch_idx, (data, target) in enumerate(data_loader):
        data, target = data.to(device=device), target.to(device=device)
        
        output = model(data)
        # loss function
#         print(torch.argmax(output,dim=1))
#         print(target)
        los = loss(output, target)
        
        loss_avg += los.item() 
        
        # do back propagation

        optimizer.zero_grad()
        los.backward()
        optimizer.step()
        
        #print
        verbose_step = len(data_loader) // 10
        
#         if verbose:
#             print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
#                 epoch, batch_idx * len(data), len(data_loader.dataset),
#                 100. * batch_idx / len(data_loader), los.item()))
    print('Train Epoch: {} tLoss: {:.6f}'.format(epoch, loss_avg))
    return loss_avg / len(data_loader)

def valid(model, data_loader):
    with torch.no_grad():
        model.eval()
        valid_loss = 0
        correct = 0
        for data, target in data_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            los = loss(output, target)
            valid_loss +=  los.item()
            pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.data.view_as(pred)).cpu().sum().item() 

        valid_loss /= len(data_loader.dataset)
        print('\nValid set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            valid_loss, correct, len(data_loader.dataset),
            100. * correct / len(data_loader.dataset)))
    return float(correct) / len(data_loader.dataset)

In [7]:
############## Build the model here ##########
class ConvNet(nn.Module):
    def __init__(self,num_classes=6):
    
        super(ConvNet, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=7, stride=2, padding=3),
            nn.BatchNorm2d(num_features=64),
            nn.ReLU(inplace=True)
        )
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2)
        
        self.conv2_1 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(num_features=64),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(num_features=64)
            
        )
        self.conv2_2 = nn.Sequential(
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(num_features=64),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(num_features=64),
            nn.ReLU(inplace=True)
        )
        self.conv3_1 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(num_features=128),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(num_features=128)  
            
        )
        self.conv1x1_1 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=1, stride=2)
        self.conv3_2 = nn.Sequential(
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(num_features=128),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(num_features=128),
            nn.ReLU(inplace=True)
        )
        self.conv4_1 = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(num_features=256),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(num_features=256)
        )
        self.conv1x1_2 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=1, stride=2)
        self.conv4_2 = nn.Sequential(
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(num_features=256),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(num_features=256),
            nn.ReLU(inplace=True)
        )
        self.conv5_1 = nn.Sequential(
            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(num_features=512),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(num_features=512)
        )
        self.conv1x1_3 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=1, stride=2)
        self.conv5_2 = nn.Sequential(
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(num_features=512),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(num_features=512),
            nn.ReLU(inplace=True)
        )
        self.avgpool = nn.AvgPool2d(kernel_size=7)
        self.flat = nn.Flatten(start_dim=1, end_dim=-1)
        self.fc = nn.Linear(in_features=512, out_features=num_classes)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        #print(x.shape)
        x = self.conv1(x) #b,112,112,64
        #print(x.shape)
        x = self.maxpool(x) #b,56,56,64
        x2_1 = self.conv2_1(x) #b,56,56,64
        x = x + x2_1 #b,56,56,64
        
        x2_2 = self.conv2_2(x) #b,56,56,64
        x = x + x2_2 #b,56,56,64
        
        x3_1 = self.conv3_1(x) #b,28,28,128
        x = self.conv1x1_1(x) #b,28,28,128
        x = x + x3_1 #b,28,28,128
        
        x3_2 = self.conv3_2(x) #b,28,28,128
        x = x + x3_2 #b,28,28,128
        
        x4_1 = self.conv4_1(x) #b,14,14,256
        x = self.conv1x1_2(x) #b,14,14,256
        x = x + x4_1 #b,14,14,256
        
        x4_2 = self.conv4_2(x) #b,14,14,256
        x = x + x4_2 #b,14,14,256
        
        x5_1 = self.conv5_1(x) #b,7,7,512
        x = self.conv1x1_3(x) #b,7,7,512
        x = x + x5_1 #b,7,7,512
        
        x5_2 = self.conv5_2(x) #b,7,7,512
        x = x + x5_2 #b,7,7,512
        
        x = self.avgpool(x) #b,1,1,512
        x = self.flat(x) #b,512
        x = self.fc(x) #b,6
        out = self.softmax(x) #b,6
        
        return out

In [8]:
# using gpu if available
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
####################  implement your optimizer ###################################
## yo can use any training methods if you want (ex:lr decay, weight decay.....)
model=ConvNet()
model.to(device=device)
lr=1e-5
# optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.5,weight_decay=5e-3)
optimizer = optim.Adam(model.parameters(),lr=lr)
# start training
epochs = 10000
acc = 0.0
loss = nn.CrossEntropyLoss()
for epoch in range(epochs):
    model.train()
    train(model, train_data_loader, optimizer, epoch, loss)
    accuracy = valid(model, val_data_loader)
#     print("-------------saving model--------------")
        # save the model
    torch.save(model, "model.pth")
#     if accuracy >= acc:
#         acc = accuracy
#         print("-------------saving model--------------")
#         # save the model
#         torch.save(model, "model.pth")

Train Epoch: 0 tLoss: 16.097955

Valid set: Average loss: 0.0345, Accuracy: 13/52 (25%)

Train Epoch: 1 tLoss: 15.683148

Valid set: Average loss: 0.0343, Accuracy: 13/52 (25%)

Train Epoch: 2 tLoss: 15.481596

Valid set: Average loss: 0.0339, Accuracy: 13/52 (25%)

Train Epoch: 3 tLoss: 14.745888

Valid set: Average loss: 0.0328, Accuracy: 17/52 (33%)

Train Epoch: 4 tLoss: 14.588204

Valid set: Average loss: 0.0317, Accuracy: 22/52 (42%)

Train Epoch: 5 tLoss: 14.374935

Valid set: Average loss: 0.0305, Accuracy: 22/52 (42%)

Train Epoch: 6 tLoss: 14.211767

Valid set: Average loss: 0.0309, Accuracy: 21/52 (40%)

Train Epoch: 7 tLoss: 13.924780

Valid set: Average loss: 0.0304, Accuracy: 22/52 (42%)

Train Epoch: 8 tLoss: 13.679127

Valid set: Average loss: 0.0295, Accuracy: 26/52 (50%)

Train Epoch: 9 tLoss: 13.445606

Valid set: Average loss: 0.0290, Accuracy: 31/52 (60%)

Train Epoch: 10 tLoss: 13.563065

Valid set: Average loss: 0.0287, Accuracy: 29/52 (56%)

Train Epoch: 11 tLos


Valid set: Average loss: 0.0243, Accuracy: 41/52 (79%)

Train Epoch: 92 tLoss: 11.512044

Valid set: Average loss: 0.0242, Accuracy: 41/52 (79%)

Train Epoch: 93 tLoss: 11.418671

Valid set: Average loss: 0.0244, Accuracy: 38/52 (73%)

Train Epoch: 94 tLoss: 11.577267

Valid set: Average loss: 0.0236, Accuracy: 41/52 (79%)

Train Epoch: 95 tLoss: 11.381599

Valid set: Average loss: 0.0241, Accuracy: 42/52 (81%)

Train Epoch: 96 tLoss: 11.478340

Valid set: Average loss: 0.0246, Accuracy: 40/52 (77%)

Train Epoch: 97 tLoss: 11.438267

Valid set: Average loss: 0.0237, Accuracy: 44/52 (85%)

Train Epoch: 98 tLoss: 11.408089

Valid set: Average loss: 0.0241, Accuracy: 42/52 (81%)

Train Epoch: 99 tLoss: 11.348695

Valid set: Average loss: 0.0234, Accuracy: 43/52 (83%)

Train Epoch: 100 tLoss: 11.405077

Valid set: Average loss: 0.0240, Accuracy: 41/52 (79%)

Train Epoch: 101 tLoss: 11.466499

Valid set: Average loss: 0.0238, Accuracy: 42/52 (81%)

Train Epoch: 102 tLoss: 11.409474

Valid 

Train Epoch: 182 tLoss: 10.791979

Valid set: Average loss: 0.0221, Accuracy: 47/52 (90%)

Train Epoch: 183 tLoss: 10.880884

Valid set: Average loss: 0.0228, Accuracy: 46/52 (88%)

Train Epoch: 184 tLoss: 10.990813

Valid set: Average loss: 0.0226, Accuracy: 46/52 (88%)

Train Epoch: 185 tLoss: 11.165888

Valid set: Average loss: 0.0226, Accuracy: 46/52 (88%)

Train Epoch: 186 tLoss: 10.871656

Valid set: Average loss: 0.0229, Accuracy: 45/52 (87%)

Train Epoch: 187 tLoss: 11.030216

Valid set: Average loss: 0.0221, Accuracy: 48/52 (92%)

Train Epoch: 188 tLoss: 10.912668

Valid set: Average loss: 0.0227, Accuracy: 45/52 (87%)

Train Epoch: 189 tLoss: 10.842141

Valid set: Average loss: 0.0226, Accuracy: 46/52 (88%)

Train Epoch: 190 tLoss: 11.093909

Valid set: Average loss: 0.0223, Accuracy: 47/52 (90%)

Train Epoch: 191 tLoss: 10.912622

Valid set: Average loss: 0.0234, Accuracy: 44/52 (85%)

Train Epoch: 192 tLoss: 10.909208

Valid set: Average loss: 0.0225, Accuracy: 45/52 (87%)



Valid set: Average loss: 0.0219, Accuracy: 47/52 (90%)

Train Epoch: 273 tLoss: 10.683091

Valid set: Average loss: 0.0220, Accuracy: 47/52 (90%)

Train Epoch: 274 tLoss: 10.595682

Valid set: Average loss: 0.0217, Accuracy: 48/52 (92%)

Train Epoch: 275 tLoss: 10.766902

Valid set: Average loss: 0.0220, Accuracy: 47/52 (90%)

Train Epoch: 276 tLoss: 10.554366

Valid set: Average loss: 0.0221, Accuracy: 47/52 (90%)

Train Epoch: 277 tLoss: 10.623693

Valid set: Average loss: 0.0220, Accuracy: 47/52 (90%)

Train Epoch: 278 tLoss: 10.731148

Valid set: Average loss: 0.0224, Accuracy: 46/52 (88%)

Train Epoch: 279 tLoss: 10.629347

Valid set: Average loss: 0.0223, Accuracy: 47/52 (90%)

Train Epoch: 280 tLoss: 10.563580

Valid set: Average loss: 0.0221, Accuracy: 47/52 (90%)

Train Epoch: 281 tLoss: 10.621252

Valid set: Average loss: 0.0226, Accuracy: 46/52 (88%)

Train Epoch: 282 tLoss: 10.458763

Valid set: Average loss: 0.0220, Accuracy: 47/52 (90%)

Train Epoch: 283 tLoss: 10.611102

Train Epoch: 363 tLoss: 10.294294

Valid set: Average loss: 0.0221, Accuracy: 47/52 (90%)

Train Epoch: 364 tLoss: 10.356730

Valid set: Average loss: 0.0222, Accuracy: 46/52 (88%)

Train Epoch: 365 tLoss: 10.506720

Valid set: Average loss: 0.0223, Accuracy: 46/52 (88%)

Train Epoch: 366 tLoss: 10.650943

Valid set: Average loss: 0.0215, Accuracy: 48/52 (92%)

Train Epoch: 367 tLoss: 10.291609

Valid set: Average loss: 0.0223, Accuracy: 46/52 (88%)

Train Epoch: 368 tLoss: 10.550708

Valid set: Average loss: 0.0228, Accuracy: 44/52 (85%)

Train Epoch: 369 tLoss: 10.374063

Valid set: Average loss: 0.0222, Accuracy: 47/52 (90%)

Train Epoch: 370 tLoss: 10.462734

Valid set: Average loss: 0.0219, Accuracy: 47/52 (90%)

Train Epoch: 371 tLoss: 10.456191

Valid set: Average loss: 0.0227, Accuracy: 45/52 (87%)

Train Epoch: 372 tLoss: 10.348825

Valid set: Average loss: 0.0217, Accuracy: 48/52 (92%)

Train Epoch: 373 tLoss: 10.346505

Valid set: Average loss: 0.0220, Accuracy: 48/52 (92%)


...


Valid set: Average loss: 0.0201, Accuracy: 52/52 (100%)

Train Epoch: 9596 tLoss: 9.532575

Valid set: Average loss: 0.0201, Accuracy: 52/52 (100%)

Train Epoch: 9597 tLoss: 9.464650

Valid set: Average loss: 0.0201, Accuracy: 52/52 (100%)

Train Epoch: 9598 tLoss: 9.471296

Valid set: Average loss: 0.0201, Accuracy: 52/52 (100%)

Train Epoch: 9599 tLoss: 9.480283

Valid set: Average loss: 0.0201, Accuracy: 52/52 (100%)

Train Epoch: 9600 tLoss: 9.482184

Valid set: Average loss: 0.0201, Accuracy: 52/52 (100%)

Train Epoch: 9601 tLoss: 9.504739

Valid set: Average loss: 0.0202, Accuracy: 52/52 (100%)

Train Epoch: 9602 tLoss: 9.469390

Valid set: Average loss: 0.0201, Accuracy: 52/52 (100%)

Train Epoch: 9603 tLoss: 9.456616

Valid set: Average loss: 0.0201, Accuracy: 52/52 (100%)

Train Epoch: 9604 tLoss: 9.458001

Valid set: Average loss: 0.0201, Accuracy: 52/52 (100%)

Train Epoch: 9605 tLoss: 9.487180

Valid set: Average loss: 0.0201, Accuracy: 52/52 (100%)

Train Epoch: 9606 tLos

Train Epoch: 9685 tLoss: 9.455209

Valid set: Average loss: 0.0201, Accuracy: 52/52 (100%)

Train Epoch: 9686 tLoss: 9.462409

Valid set: Average loss: 0.0201, Accuracy: 52/52 (100%)

Train Epoch: 9687 tLoss: 9.455263

Valid set: Average loss: 0.0201, Accuracy: 52/52 (100%)

Train Epoch: 9688 tLoss: 9.455228

Valid set: Average loss: 0.0201, Accuracy: 52/52 (100%)

Train Epoch: 9689 tLoss: 9.455404

Valid set: Average loss: 0.0201, Accuracy: 52/52 (100%)

Train Epoch: 9690 tLoss: 9.470714

Valid set: Average loss: 0.0201, Accuracy: 52/52 (100%)

Train Epoch: 9691 tLoss: 9.455344

Valid set: Average loss: 0.0201, Accuracy: 52/52 (100%)

Train Epoch: 9692 tLoss: 9.457227

Valid set: Average loss: 0.0201, Accuracy: 52/52 (100%)

Train Epoch: 9693 tLoss: 9.455345

Valid set: Average loss: 0.0201, Accuracy: 52/52 (100%)

Train Epoch: 9694 tLoss: 9.467090

Valid set: Average loss: 0.0201, Accuracy: 52/52 (100%)

Train Epoch: 9695 tLoss: 9.455217

Valid set: Average loss: 0.0201, Accuracy: 52


Valid set: Average loss: 0.0201, Accuracy: 52/52 (100%)

Train Epoch: 9775 tLoss: 9.530184

Valid set: Average loss: 0.0205, Accuracy: 51/52 (98%)

Train Epoch: 9776 tLoss: 9.554030

Valid set: Average loss: 0.0201, Accuracy: 52/52 (100%)

Train Epoch: 9777 tLoss: 9.470466

Valid set: Average loss: 0.0201, Accuracy: 52/52 (100%)

Train Epoch: 9778 tLoss: 9.555099

Valid set: Average loss: 0.0201, Accuracy: 52/52 (100%)

Train Epoch: 9779 tLoss: 9.497611

Valid set: Average loss: 0.0201, Accuracy: 52/52 (100%)

Train Epoch: 9780 tLoss: 9.496835

Valid set: Average loss: 0.0201, Accuracy: 52/52 (100%)

Train Epoch: 9781 tLoss: 9.551529

Valid set: Average loss: 0.0201, Accuracy: 52/52 (100%)

Train Epoch: 9782 tLoss: 9.461864

Valid set: Average loss: 0.0201, Accuracy: 52/52 (100%)

Train Epoch: 9783 tLoss: 9.468925

Valid set: Average loss: 0.0201, Accuracy: 52/52 (100%)

Train Epoch: 9784 tLoss: 9.471177

Valid set: Average loss: 0.0201, Accuracy: 52/52 (100%)

Train Epoch: 9785 tLoss

Train Epoch: 9864 tLoss: 9.456722

Valid set: Average loss: 0.0201, Accuracy: 52/52 (100%)

Train Epoch: 9865 tLoss: 9.457827

Valid set: Average loss: 0.0201, Accuracy: 52/52 (100%)

Train Epoch: 9866 tLoss: 9.552602

Valid set: Average loss: 0.0201, Accuracy: 52/52 (100%)

Train Epoch: 9867 tLoss: 9.466422

Valid set: Average loss: 0.0201, Accuracy: 52/52 (100%)

Train Epoch: 9868 tLoss: 9.457035

Valid set: Average loss: 0.0201, Accuracy: 52/52 (100%)

Train Epoch: 9869 tLoss: 9.479010

Valid set: Average loss: 0.0201, Accuracy: 52/52 (100%)

Train Epoch: 9870 tLoss: 9.456244

Valid set: Average loss: 0.0201, Accuracy: 52/52 (100%)

Train Epoch: 9871 tLoss: 9.461611

Valid set: Average loss: 0.0201, Accuracy: 52/52 (100%)

Train Epoch: 9872 tLoss: 9.471421

Valid set: Average loss: 0.0201, Accuracy: 52/52 (100%)

Train Epoch: 9873 tLoss: 9.469633

Valid set: Average loss: 0.0201, Accuracy: 52/52 (100%)

Train Epoch: 9874 tLoss: 9.455692

Valid set: Average loss: 0.0201, Accuracy: 52


Valid set: Average loss: 0.0204, Accuracy: 51/52 (98%)

Train Epoch: 9954 tLoss: 9.491536

Valid set: Average loss: 0.0204, Accuracy: 51/52 (98%)

Train Epoch: 9955 tLoss: 9.476595

Valid set: Average loss: 0.0201, Accuracy: 52/52 (100%)

Train Epoch: 9956 tLoss: 9.484913

Valid set: Average loss: 0.0201, Accuracy: 52/52 (100%)

Train Epoch: 9957 tLoss: 9.562905

Valid set: Average loss: 0.0204, Accuracy: 51/52 (98%)

Train Epoch: 9958 tLoss: 9.479554

Valid set: Average loss: 0.0201, Accuracy: 52/52 (100%)

Train Epoch: 9959 tLoss: 9.556047

Valid set: Average loss: 0.0201, Accuracy: 52/52 (100%)

Train Epoch: 9960 tLoss: 9.462840

Valid set: Average loss: 0.0204, Accuracy: 51/52 (98%)

Train Epoch: 9961 tLoss: 9.489568

Valid set: Average loss: 0.0201, Accuracy: 52/52 (100%)

Train Epoch: 9962 tLoss: 9.464875

Valid set: Average loss: 0.0201, Accuracy: 52/52 (100%)

Train Epoch: 9963 tLoss: 9.471001

Valid set: Average loss: 0.0201, Accuracy: 52/52 (100%)

Train Epoch: 9964 tLoss: 9

In [10]:
test_transform = transforms.Compose([transforms.Resize(img_size),
                                     transforms.ToTensor()])

test_data = torchvision.datasets.ImageFolder(TEST_DATA_PATH,
                                            transform=test_transform
                                            )
test_data_loader  = data.DataLoader(test_data, batch_size=64, shuffle=False, num_workers=2)

In [11]:
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# load the model so that you don't need to train the model again
test_model = torch.load("model.pth").to(device)

In [12]:
def test(model,data_loader):
    with torch.no_grad():
        model.eval()
        valid_loss = 0
        correct = 0
        bs = test_data_loader.batch_size
        result = []
        for i, (data, target) in enumerate(test_data_loader):
            data, target = data.to(device), target.to(device)
            output = model(data)
            #print(output)
            pred = torch.argmax(output,dim=1)                  # get the index of the max log-probability
            #print(pred.shape)
            for lll in range(len(pred)):
                pred[lll] = pallet[pred[lll]]
            arr = pred.data.cpu().numpy()
            for j in range(pred.size()[0]):
                file_name = test_data.samples[i*bs+j][0].split('/')[-1]
                #print(file_name,pred[j].cpu().numpy())
                result.append((file_name,pred[j].cpu().numpy()))
    return result

In [13]:
result = test(test_model,test_data_loader)

# Write results to csv

In [14]:
df = pd.DataFrame(result, columns=["Id","Category"])
df.to_csv('ML-test-predict.csv',index=False)
# df_new = pd.read_csv('ML-test-predict.csv',index_col=[0])
print("dump file...")

dump file...
